<a href="https://colab.research.google.com/github/srivatsan88/YouTubeLI/blob/master/Topic_Modeling_using_NMF_scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('punkt')

In [ ]:
df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')

In [ ]:
df

In [ ]:
df['Product'].value_counts()

In [ ]:
df['Company'].value_counts()

In [ ]:
complaints_df=df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints'})

In [ ]:
pd.set_option('display.max_colwidth', -1)
complaints_df

In [ ]:
X_train, X_hold = train_test_split(complaints_df, test_size=0.3, random_state=111)

In [ ]:
X_train['Product'].value_counts()

In [ ]:
#stemmer = PorterStemmer()
stemmer = nltk.stem.SnowballStemmer('english')
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

In [ ]:
def tokenize(text):
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2 and len(re.sub('\d+', '', word.strip('Xx/'))) > 3) ] 
   tokens = map(str.lower, tokens)
   stems = [stemmer.stem(item) for item in tokens if (item not in stop_words)]
   return stems

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words=None, max_df=0.75, max_features=1000, lowercase=False, ngram_range=(1,2))
tfidf_vectors = vectorizer.fit_transform(X_train.complaints) 

In [ ]:
tfidf_vectors.A

In [ ]:
vectorizer.get_feature_names()

In [ ]:
clf = decomposition.NMF(n_components=6, random_state=111)

W1 = clf.fit_transform(tfidf_vectors)
H1 = clf.components_

In [ ]:
H1

array([[0.00000000e+00, 2.26701694e-01, 5.67180774e-02, ...,
        1.04962564e-01, 4.93310687e-02, 2.36510109e-01],
       [9.02557623e-02, 5.16984659e-03, 6.72853708e-03, ...,
        6.30409582e-02, 3.56037922e-02, 1.66265209e-01],
       [7.23475845e-03, 1.17355438e-01, 1.45066546e-02, ...,
        9.38266155e-02, 4.20839489e-02, 9.72722832e-02],
       [9.27737536e-02, 1.50914117e-01, 2.39515858e-02, ...,
        6.61443857e-02, 1.55528206e-02, 5.20053002e-01],
       [0.00000000e+00, 5.84106407e-05, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.02964646e-03, 8.09717204e-02, 2.67704104e-02, ...,
        8.54121270e-02, 1.32456081e-02, 1.66357867e-01]])

In [ ]:
W1

array([[0.05224125, 0.00941627, 0.        , 0.        , 0.        ,
        0.        ],
       [0.03875709, 0.        , 0.05520863, 0.01873516, 0.00121965,
        0.04867738],
       [0.        , 0.04892786, 0.0223263 , 0.        , 0.00657403,
        0.00129874],
       ...,
       [0.02200383, 0.        , 0.0426682 , 0.00044996, 0.        ,
        0.0314196 ],
       [0.        , 0.02779387, 0.00191482, 0.00111531, 0.00191333,
        0.02376908],
       [0.        , 0.        , 0.        , 0.07995555, 0.        ,
        0.        ]])

In [ ]:
num_words=15

vocab = np.array(vectorizer.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [ ]:
topics

['call told phone number would receiv said back ask time contact compani email inform work',
 'debt report credit collect credit report compani valid account disput remov agenc inform letter request provid',
 'account bank check close deposit open money fund check account bank america america close account chase branch transact',
 'payment loan mortgag month late interest paid make made amount servic year home appli escrow',
 'ident theft theft ident victim ident victim belong debt belong affidavit report attach legal pleas result debt fraudul',
 'card credit credit card charg purchas capit balanc disput citi use transact fraud interest chase merchant']

In [ ]:
colnames = ["Topic" + str(i) for i in range(clf.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [ ]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.05,0.01,0.00,0.00,0.00,0.00,0
Doc1,0.04,0.00,0.06,0.02,0.00,0.05,2
Doc2,0.00,0.05,0.02,0.00,0.01,0.00,1
Doc3,0.03,0.00,0.01,0.00,0.00,0.01,0
Doc4,0.01,0.00,0.04,0.00,0.00,0.01,2
...,...,...,...,...,...,...,...
Doc40212,0.01,0.00,0.00,0.00,0.00,0.06,5
Doc40213,0.00,0.01,0.05,0.01,0.00,0.02,2
Doc40214,0.02,0.00,0.04,0.00,0.00,0.03,2
Doc40215,0.00,0.03,0.00,0.00,0.00,0.02,1


In [ ]:
X_train.head()

complaints  ...                          Company
30936  Throughout XX/XX/2019, I have received multiple calls from this company. I have also asked them to not call me or anyone associated with me. They have called my mother in law way too many times asking for my partner. They have called my partner multiple times asking for me. I'm not sure what kind of operation is being ran there, but it needs to stop. I'm giving notice to cease all contact with me or anyone else except the creditor about this claimed debt. Only contact me in writing, not by phone. Thank you.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
WHold = clf.transform(vectorizer.transform(X_hold.complaints[:5]))

In [ ]:
colnames = ["Topic" + str(i) for i in range(clf.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold[:5].complaints))]
df_doc_topic = pd.DataFrame(np.round(WHold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [ ]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.01,0.00,0.10,0.00,0.0,0.00,2
Doc1,0.01,0.02,0.02,0.04,0.0,0.00,3
Doc2,0.00,0.00,0.04,0.01,0.0,0.04,2
Doc3,0.01,0.00,0.00,0.06,0.0,0.00,3
Doc4,0.00,0.00,0.01,0.04,0.0,0.04,3


In [ ]:
X_hold.head()

,complaints,Product,Company
30060,"I have a business checking account at BB & T. On XX/XX/2019, I attempted to deposit a check into my account and I received a message stating that I was over my monthly mobile deposit limit. I was confused because it was the first of the month and I had not deposited any checks since the previous month. I called BB & T and they said that I couldnt deposit checks into business accounts via the mobile app even though I had done that before. \n\nI was instructed to open a personal account, into which I could deposit checks via the mobile app. I was told that if I opened the account online I would have immediate access, that I could link my personal and business accounts, and immediately be able to transfer money between them. \n\nOn XX/XX/XXXX, I opened my personal account online. Though I successfully opened online, I did not have online access as I had been promised. Because I was traveling in an area where there were no BB & T branches, I could not go into a branch until XX/XX/2019. In the intervening time, my business account became overdrawn. I had the money in my personal account to bring it back into the black, but BB & T could not make the transfer until I went into a branch. During this time, I incurred an astonishing {$320.00} in overdraft fees in my business account because I had no way of transferring money between the two accounts. \n\nWhen I went into the branch, I met with XXXX XXXX in XXXX XXXX County, Florida. She investigated the accounts and told me she would link them and that I would have online access in XXXX hours. The online access still never material, I was not able to transfer money and I received an additional {$36.00} overdraft fee. This brought the total to {$360.00} in overdraft fees. \n\nI had only opened the second account on BB & Ts advice ; I could have had my customer send a bank wire directly into business account in the first week of XXXX to keep the account positive. My TOTAL deposits between the two accounts was always positive. \n\nI requested a refund and was told they would not refund any of the fees.",Checking or savings account,BB&T CORPORATION
53473,"To who it may concern, My concern is regarding Shellpoint Mortgage Servicing Company. This company has received my monthly mortgage payments and have failed to account it as payments received. My bank has sent me proof the account was cleared and paid. This company is trying to foreclosed on my house and has sent me a letter indicating they sent my file to their Loss Mitigation dept. The person who they assigned it to is XXXX XXXX XXXX x XXXX. Their corporate number is XXXX. \n\nI have can send proof of my claim from above should you need it. Please assist.",Mortgage,"Shellpoint Partners, LLC"
35879,I contacted XXXX about fraudulent charges that were made on my account and the customer service representative told me that they wouldnt be able to issue anew card or remove my fraudulent charges since the account was closed due to non-payment. I was unaware of this these charging that were made.. To my knowledge I didnt owe a payment because the account wasnt being used.,Credit card or prepaid card,"EQUIFAX, INC."
20993,"I first applied for the Fedloan Serving program in XXXX in hopes of getting loan forgiveness since I've been paying on my student loans since late XXXX or early XXXX and I have been a public servant since XXXX, XXXX. I have never missed a payment to any of the various loan companies that owned my loans. The Fedloan Program turned me down in XXXX, saying my loans, which were originally Federal Stafford Loans, did not qualify me for the Program. In XXXX, XXXX, I was solicited by XXXX XXXX and told I could get into this program to reduce my loan payments. I was charged {$690.00} by XXXX XXXX to get me into the Fedloan Serving program. My loan payments went down to $ XXXX/month for the first year. In XXXX, XXXX, Fedloan Servicing informed me via email that I needed to recertify my "" income-drive